In [1]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at d:\downloads\python3.11\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install dlib

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at d:\downloads\python3.11\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import sys
print(sys.version)

3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]


In [4]:
pwd

'd:\\Face_reco_project'

In [5]:
pip install dlib-19.24.1-cp311-cp311-win_amd64.whl

Processing d:\face_reco_project\dlib-19.24.1-cp311-cp311-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at d:\downloads\python3.11\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'D:\\Face_reco_project\\dlib-19.24.1-cp311-cp311-win_amd64.whl'



In [6]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at d:\downloads\python3.11\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [7]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import pickle
from flask import Flask, request, render_template

In [8]:
pp = Flask(__name__)

def load_model():
    with open('D://face rec project//face recognition//face recognition//attendance.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

def faceencoding(images):
    encodinglist = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodinglist.append(encode)
    return encodinglist


In [9]:
def is_entry_exists(name):
    with open('D://face rec project//face recognition//face recognition//att.csv', 'r') as f:
        for line in f:
            entry = line.split(',')
            if entry[0] == name:
                return True
    return False

def attendance(name):
    if not is_entry_exists(name):
        with open('D://face rec project//face recognition//face recognition//att.csv', 'a') as f:
            time_row = datetime.now()
            tstr = time_row.strftime('%H:%M:%S')
            dstr = time_row.strftime('%d/%m/%y')
            f.write(f'{name},{tstr},{dstr}\n')


In [10]:
 path = 'D://face rec project//face recognition//face recognition//photos'

images = []
personname = []
mylist = os.listdir(path)
print(mylist)

['Eileen.jpeg', 'Papai.png', 'Pradip.png', 'Raja.jpeg', 'Tapan Mukherjee.jpeg']


In [11]:
def faceencoding(images):
    encodinglist = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(img)
        if len(face_encodings) > 0:
            encode = face_encodings[0]
            encodinglist.append(encode)
    return encodinglist

In [12]:
for i in mylist:
    current_image = cv2.imread(f'{path}/{i}')
    images.append(current_image)
    personname.append(os.path.splitext(i)[0])
print(personname)
encodelistknown = faceencoding(images)

['Eileen', 'Papai', 'Pradip', 'Raja', 'Tapan Mukherjee']


In [13]:
cap = cv2.VideoCapture(0)
while True:
        ret, frame = cap.read()
        small_frame = cv2.resize(frame, (0, 0), None, 0.25, 0.25)
        small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        names_detected = []
        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(encodelistknown, face_encoding)
            face_distances = face_recognition.face_distance(encodelistknown, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = personname[best_match_index].upper()
                names_detected.append(name)
                top, right, bottom, left = face_location
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        for name in names_detected:
            attendance(name)
        cv2.imshow("camera", frame)
        if cv2.waitKey(1) == 32:
            break
cap.release()
cv2.destroyAllWindows()